# Notice:
To train Dreambooth Stable Diffusion requires a recommended minimum of 16 GB of GPU RAM. Trying to train Dreambooth with less will trigger an Out of Memory error. This Notebook is defaulted to set up to run on the RTX5000 GPU, which requires a paid account on Gradient. 

Change the URL at the top where it says "machine?=RTX5000" to any GPU with at least 16 GB of RAM to run training. The P6000, V100, V100-32G, RTX5000, A4000, A5000, A100, and A100-80G powered machines will all be able to run this training. 

You can do so by changing the following URL where it says "(YOUR-GPU-CHOICE)":

 https://console.paperspace.com/github/gradient-ai/dreambooth-stable-diffusion/blob/main/sd_dreambooth_gradient.ipynb?machine=(YOUR-GPU-CHOICE)


# Dreambooth fine-tuning for Stable Diffusion using d🧨ffusers with Gradient Notebooks

This notebook shows how to "teach" Stable Diffusion a new concept via Dreambooth using 🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers). 

![Dreambooth Example](https://dreambooth.github.io/DreamBooth_files/teaser_static.jpg)
_By using just 3-5 images you can teach new concepts to Stable Diffusion and personalize the model on your own images_ 

Differently from Textual Inversion, this approach trains the whole model, which can yield better results to the cost of bigger models.

For a general introduction to the Stable Diffusion model please refer to this [Gradient Notebook](https://console.paperspace.com/github/gradient-ai/stable-diffusion/blob/main/stable-diffusion-notebook.ipynb?machine=A4000).



## Installation

In [1]:
#@title Install the required libs
!pip install -qq accelerate tensorboard ftfy
!pip install -qq --upgrade diffusers[torch] --no-cache-dir
!pip install -qq -U transformers
!pip install -qq bitsandbytes
!pip install gradio
!mkdir inputs
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# Loading in the Stable Diffusion v1-5 models

To make accessing the Stable Diffusion models easy and not take up any storage, we have added the Stable Diffusion models v1-5 as mountable public datasets! To access the models this way, simply navigate to the "Data Sources" tab using the navigator on the far left of the page. Then click "Public" to switch into the Gradient Public Datasets, and scroll down until you find "stable-diffusion" near the bottom of the list. Then, click "mount" to make them accessible from the "datasets" directory. This directory is in the root folder, so access it with the path `../datasets/stable-diffusion-diffusers/stable-diffusion-v1-5`

> Note that these public dataset files will not count toward the file storage limits 


<video controls src="assets/stab-upload.mp4" />

# Alternate access: log in to HuggingFace for online access to models

In order to access the models from CompVis for Stable Diffusion, you must follow three steps:

1. You must acknowledge and agree to their user requirements and license for their models. you can do so by reading the instructions found on this page: https://huggingface.co/runwayml/stable-diffusion-v1-5

2. You must login to Huggingface, and then create and retrieve an access token (found here: https://huggingface.co/settings/tokens)

3. Finally, replace the segment of the cell below `<your_huggingface_token>` with your own token, and run the cell. 

If you follow these steps, you will be able to access the model for free!

If you are using this method, be sure to change the path to `runwayml/stable-diffusion-v1-5`



In [2]:
# !wget https://raw.githubusercontent.com/gradient-ai/stable-diffusion/main/login.py
# !python login.py --token <your_huggingface_token>

## Imports and setup

In [1]:
#@title Import required libraries
import argparse
import itertools
import math

import os
from contextlib import nullcontext
import random

import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
from torch.utils.data import Dataset

import PIL
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from diffusers import AutoencoderKL, DDPMScheduler, PNDMScheduler, StableDiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer

import bitsandbytes as bnb


# Helper function

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

## Settings for teaching your new concept

When setting up your data, you can use the demo data provided below to create your first inversion. 

### Display your video output in markdown
If you would like to use your own images, hash out the cells below and add them to the `inputs` directory.

<video controls src="assets/upload.mp4"/>


In [ ]:
#@markdown Add here the URLs to the images of the concept you are adding. 3-5 should be fine
urls = [
      "https://huggingface.co/datasets/valhalla/images/resolve/main/2.jpeg",
      "https://huggingface.co/datasets/valhalla/images/resolve/main/3.jpeg",
      "https://huggingface.co/datasets/valhalla/images/resolve/main/5.jpeg",
      "https://huggingface.co/datasets/valhalla/images/resolve/main/6.jpeg",
      ## You can add additional images here
      ]

In [ ]:
# @title Setup and check the images you have just added
import requests
import glob
from io import BytesIO

def download_image(url):
  try:
    response = requests.get(url)
  except:
    return None
  return Image.open(BytesIO(response.content)).convert("RGB")

images = list(filter(None,[download_image(url) for url in urls]))
save_path = "./inputs"
if not os.path.exists(save_path):
  os.mkdir(save_path)
[image.save(f"{save_path}/{i}.jpeg") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

## Set training variables

In [3]:
#@title Settings for your newly created concept
#@markdown `instance_prompt` is a prompt that should contain a good description of what your object or style is, together with the initializer word `sks`  
instance_prompt = "a photo of sks toy" #@param {type:"string"}
#@markdown Check the `prior_preservation` option if you would like class of the concept (e.g.: toy, dog, painting) is guaranteed to be preserved. This increases the quality and helps with generalization at the cost of training time
prior_preservation = False #@param {type:"boolean"}
prior_preservation_class_prompt = "a photo of a cat toy" #@param {type:"string"}
class_prompt=prior_preservation_class_prompt

# Set your data folder path 
prior_preservation_class_folder = './inputs'
class_data_root=prior_preservation_class_folder

num_class_images = len(os.listdir(prior_preservation_class_folder))
sample_batch_size = 4
prior_loss_weight = 0.5

#### Advanced settings for prior preservation (optional)

In [ ]:
!mkdir class_images

In [50]:
#@markdown If the `prior_preservation_class_folder` is empty, images for the class will be generated with the class prompt. Otherwise, fill this folder with images of items on the same class as your concept (but not images of the concept itself)
prior_preservation_class_folder = "./class_images" #@param {type:"string"}
class_data_root=prior_preservation_class_folder

num_class_images = len(os.listdir(prior_preservation_class_folder)) #@param {type: "number"}
sample_batch_size = 2
#@markdown `prior_preservation_weight` determins how strong the class for prior preservation should be 
prior_loss_weight = 1 #@param {type: "number"}

## Instantiate Dataset classes

In [ ]:
#@title Setup the Classes
from pathlib import Path
from torchvision import transforms

class DreamBoothDataset(Dataset):
    def __init__(
        self,
        instance_data_root,
        instance_prompt,
        tokenizer,
        class_data_root=None,
        class_prompt=None,
        size=512,
        center_crop=False,
    ):
        self.size = size
        self.center_crop = center_crop
        self.tokenizer = tokenizer

        self.instance_data_root = Path(instance_data_root)
        if not self.instance_data_root.exists():
            raise ValueError("Instance images root doesn't exists.")

        self.instance_images_path = list(Path(instance_data_root).iterdir())
        self.num_instance_images = len(self.instance_images_path)
        self.instance_prompt = instance_prompt
        self._length = self.num_instance_images

        if class_data_root is not None:
            self.class_data_root = Path(class_data_root)
            self.class_data_root.mkdir(parents=True, exist_ok=True)
            self.class_images_path = list(Path(class_data_root).iterdir())
            self.num_class_images = len(self.class_images_path)
            self._length = max(self.num_class_images, self.num_instance_images)
            self.class_prompt = class_prompt
        else:
            self.class_data_root = None

        self.image_transforms = transforms.Compose(
            [
                transforms.Resize(size, interpolation=transforms.InterpolationMode.BILINEAR),
                transforms.CenterCrop(size) if center_crop else transforms.RandomCrop(size),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5]),
            ]
        )

    def __len__(self):
        return self._length

    def __getitem__(self, index):
        example = {}
        instance_image = Image.open(self.instance_images_path[index % self.num_instance_images])
        if not instance_image.mode == "RGB":
            instance_image = instance_image.convert("RGB")
        example["instance_images"] = self.image_transforms(instance_image)
        example["instance_prompt_ids"] = self.tokenizer(
            self.instance_prompt,
            padding="do_not_pad",
            truncation=True,
            max_length=self.tokenizer.model_max_length,
        ).input_ids

        if self.class_data_root:
            class_image = Image.open(self.class_images_path[index % self.num_class_images])
            if not class_image.mode == "RGB":
                class_image = class_image.convert("RGB")
            example["class_images"] = self.image_transforms(class_image)
            example["class_prompt_ids"] = self.tokenizer(
                self.class_prompt,
                padding="do_not_pad",
                truncation=True,
                max_length=self.tokenizer.model_max_length,
            ).input_ids
        
        return example


class PromptDataset(Dataset):
    def __init__(self, prompt, num_samples):
        self.prompt = prompt
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples

    def __getitem__(self, index):
        example = {}
        example["prompt"] = self.prompt
        example["index"] = index
        return example

## Optional: Generate Class Images 

In [5]:
#@title Generate Class Images
import gc
if(prior_preservation):
    class_images_dir = Path(class_data_root)
    if not class_images_dir.exists():
        class_images_dir.mkdir(parents=True)
    cur_class_images = len(list(class_images_dir.iterdir()))

    if cur_class_images < num_class_images:
        pipeline = StableDiffusionPipeline.from_pretrained(
            pretrained_model_name_or_path, revision="fp16", torch_dtype=torch.float16
        ).to("cuda")
        pipeline.enable_attention_slicing()
        pipeline.set_progress_bar_config(disable=True)

        num_new_images = num_class_images - cur_class_images
        print(f"Number of class images to sample: {num_new_images}.")

        sample_dataset = PromptDataset(class_prompt, num_new_images)
        sample_dataloader = torch.utils.data.DataLoader(sample_dataset, batch_size=sample_batch_size)

        for example in tqdm(sample_dataloader, desc="Generating class images"):
            images = pipeline(example["prompt"]).images

            for i, image in enumerate(images):
                image.save(class_images_dir / f"{example['index'][i] + cur_class_images}.jpg")
        pipeline = None
        gc.collect()
        del pipeline
        with torch.no_grad():
          torch.cuda.empty_cache()

## Load in your model checkpoints

You can either use the Stable Diffusion v1-5 or v2 checkpoints here. Use the 2 cells below for v1-5, and the 2 cells below them for v2.

### Stable Diffusion v1-5 or v2

Hash out the line below you would not like to use. Default is v2.

In [ ]:
#@title Load the Stable Diffusion model
# Load models and create wrapper for stable diffusion
#@markdown `pretrained_model_name_or_path` which Stable Diffusion checkpoint you want to use
## Use local files

## V1-5
pretrained_model_name_or_path = "../datasets/stable-diffusion-diffusers/stable-diffusion-v1-5/" #@param {type:"string"}

##v2
# pretrained_model_name_or_path = '../datasets/stable-diffusion-diffusers-v2/stable-diffusion-2/'



## Download online files
#@markdown Please read and, if you agree, accept the LICENSE [here](https://huggingface.co/runwayml/stable-diffusion-v1-5) if you see an error
# pretrained_model_name_or_path = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}

In [ ]:
text_encoder = CLIPTextModel.from_pretrained(
    pretrained_model_name_or_path+'text_encoder', local_files_only = True,
)
vae = AutoencoderKL.from_pretrained(
    pretrained_model_name_or_path+"vae", local_files_only = True, 
)
unet = UNet2DConditionModel.from_pretrained(
    pretrained_model_name_or_path+"unet", local_files_only = True,
)
tokenizer = CLIPTokenizer.from_pretrained(
    pretrained_model_name_or_path+"tokenizer", local_files_only = True)

## Set up args for training and output directory as save_path

In [ ]:
# We create an outputs directory here, but you can use any directory by changing `save_path` in the cell below
!mkdir outputs

In [15]:
#@title Setting up all training args
save_path = 'outputs'
from argparse import Namespace
if pretrained_model_name_or_path == '../datasets/stable-diffusion-diffusers-v2/stable-diffusion-2/':
    args = Namespace(
        pretrained_model_name_or_path=pretrained_model_name_or_path,
        resolution=768,
        center_crop=True,
        instance_data_dir=save_path,
        instance_prompt=instance_prompt,
        learning_rate=5e-06,
        max_train_steps=500,
        train_batch_size=1,
        gradient_accumulation_steps=2,
        max_grad_norm=1.0,
        mixed_precision="no", # set to "fp16" for mixed-precision training.
        gradient_checkpointing=True, # set this to True to lower the memory usage.
        use_8bit_adam=True, # use 8bit optimizer from bitsandbytes
        seed=34354,
        with_prior_preservation=prior_preservation, 
        prior_loss_weight=prior_loss_weight,
        sample_batch_size=2,
        class_data_dir=prior_preservation_class_folder, 
        class_prompt=None, 
        num_class_images=num_class_images, 
        output_dir="dreambooth-concept",
    )
else:
    args = Namespace(
        pretrained_model_name_or_path=pretrained_model_name_or_path,
        resolution=512,
        center_crop=True,
        instance_data_dir=save_path,
        instance_prompt=instance_prompt,
        learning_rate=5e-06,
        max_train_steps=500,
        train_batch_size=1,
        gradient_accumulation_steps=2,
        max_grad_norm=1.0,
        mixed_precision="no", # set to "fp16" for mixed-precision training.
        gradient_checkpointing=True, # set this to True to lower the memory usage.
        use_8bit_adam=True, # use 8bit optimizer from bitsandbytes
        seed=34354,
        with_prior_preservation=prior_preservation, 
        prior_loss_weight=prior_loss_weight,
        sample_batch_size=2,
        class_data_dir=prior_preservation_class_folder, 
        class_prompt=None, 
        num_class_images=num_class_images, 
        output_dir="dreambooth-concept",
    )

## Define training function with accelerate

In [16]:
#@title Training function
from accelerate.utils import set_seed
def training_function(text_encoder, vae, unet):
    logger = get_logger(__name__)

    accelerator = Accelerator(
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        mixed_precision=args.mixed_precision,
    )

    set_seed(args.seed)

    if args.gradient_checkpointing:
        unet.enable_gradient_checkpointing()

    # Use 8-bit Adam for lower memory usage or to fine-tune the model in 16GB GPUs
    if args.use_8bit_adam:
        optimizer_class = bnb.optim.AdamW8bit
    else:
        optimizer_class = torch.optim.AdamW

    optimizer = optimizer_class(
        unet.parameters(),  # only optimize unet
        lr=args.learning_rate,
    )

    noise_scheduler = DDPMScheduler(
        beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000
    )
    
    train_dataset = DreamBoothDataset(
        instance_data_root='inputs',
        instance_prompt=args.instance_prompt,
        class_data_root=args.class_data_dir if args.with_prior_preservation else None,
        class_prompt=class_prompt,
        tokenizer=tokenizer,
        size=args.resolution,
        center_crop=args.center_crop,
    )

    def collate_fn(examples):
        input_ids = [example["instance_prompt_ids"] for example in examples]
        pixel_values = [example["instance_images"] for example in examples]

        # concat class and instance examples for prior preservation
        if args.with_prior_preservation:
            input_ids += [example["class_prompt_ids"] for example in examples]
            pixel_values += [example["class_images"] for example in examples]

        pixel_values = torch.stack(pixel_values)
        pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()

        input_ids = tokenizer.pad({"input_ids": input_ids}, padding=True, return_tensors="pt").input_ids

        batch = {
            "input_ids": input_ids,
            "pixel_values": pixel_values,
        }
        return batch
    
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.train_batch_size, shuffle=True, collate_fn=collate_fn
    )

    unet, optimizer, train_dataloader = accelerator.prepare(unet, optimizer, train_dataloader)

    # Move text_encode and vae to gpu
    text_encoder.to(accelerator.device)
    vae.to(accelerator.device)

    # We need to recalculate our total training steps as the size of the training dataloader may have changed.
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
    num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)
  
    # Train!
    total_batch_size = args.train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Instantaneous batch size per device = {args.train_batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {args.max_train_steps}")
    # Only show the progress bar once on each machine.
    progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
    progress_bar.set_description("Steps")
    global_step = 0

    for epoch in range(num_train_epochs):
        unet.train()
        for step, batch in enumerate(train_dataloader):
            with accelerator.accumulate(unet):
                # Convert images to latent space
                with torch.no_grad():
                    latents = vae.encode(batch["pixel_values"]).latent_dist.sample()
                    latents = latents * 0.18215

                # Sample noise that we'll add to the latents
                noise = torch.randn(latents.shape).to(latents.device)
                bsz = latents.shape[0]
                # Sample a random timestep for each image
                timesteps = torch.randint(
                    0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device
                ).long()

                # Add noise to the latents according to the noise magnitude at each timestep
                # (this is the forward diffusion process)
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                # Get the text embedding for conditioning
                with torch.no_grad():
                    encoder_hidden_states = text_encoder(batch["input_ids"])[0]

                # Predict the noise residual
                noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

                if args.with_prior_preservation:
                    # Chunk the noise and noise_pred into two parts and compute the loss on each part separately.
                    noise_pred, noise_pred_prior = torch.chunk(noise_pred, 2, dim=0)
                    noise, noise_prior = torch.chunk(noise, 2, dim=0)

                    # Compute instance loss
                    loss = F.mse_loss(noise_pred, noise, reduction="none").mean([1, 2, 3]).mean()

                    # Compute prior loss
                    prior_loss = F.mse_loss(noise_pred_prior, noise_prior, reduction="none").mean([1, 2, 3]).mean()

                    # Add the prior loss to the instance loss.
                    loss = loss + args.prior_loss_weight * prior_loss
                else:
                    loss = F.mse_loss(noise_pred, noise, reduction="none").mean([1, 2, 3]).mean()

                accelerator.backward(loss)
                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(unet.parameters(), args.max_grad_norm)
                optimizer.step()
                optimizer.zero_grad()

            # Checks if the accelerator has performed an optimization step behind the scenes
            if accelerator.sync_gradients:
                progress_bar.update(1)
                global_step += 1

            logs = {"loss": loss.detach().item()}
            progress_bar.set_postfix(**logs)

            if global_step >= args.max_train_steps:
                break

        accelerator.wait_for_everyone()
    
    # Create the pipeline using using the trained modules and save it.
    if accelerator.is_main_process:
        pipeline = StableDiffusionPipeline(
            text_encoder=text_encoder,
            vae=vae,
            unet=accelerator.unwrap_model(unet),
            tokenizer=tokenizer,
            scheduler=PNDMScheduler(
                beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", skip_prk_steps=True
            ),
            safety_checker=StableDiffusionSafetyChecker.from_pretrained("CompVis/stable-diffusion-safety-checker"),
            feature_extractor=CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32"),
        )
        pipeline.save_pretrained(args.output_dir)

# Teach the model the new concept (fine-tuning with Dreambooth)
Execute this this sequence of cells to run the training process. The whole process may take from 15 min to 2 hours. (Open this block if you are interested in how this process works under the hood or if you want to change advanced training settings or hyperparameters)

In [ ]:
#@title Run training
import accelerate
accelerate.notebook_launcher(training_function, args=(text_encoder, vae, unet), num_processes =1)
with torch.no_grad():
    torch.cuda.empty_cache()

# Set up the inference pipeline

Use this pipeline to generate images from your new model

In [19]:
#@title Set up the pipeline 
try:
    pipe
except NameError:
    pipe = StableDiffusionPipeline.from_pretrained(
        args.output_dir,
        torch_dtype=torch.float16,
    ).to("cuda")

In [20]:
#@title Run the Stable Diffusion pipeline with interactive UI Demo on Gradio
#@markdown Run this cell to get an interactive demo where you can run the model using Gradio

#@markdown ![](https://i.imgur.com/2ACLWu2.png)
import gradio as gr

def inference(prompt, num_samples):
    all_images = [] 
    images = pipe(prompt, num_images_per_prompt=num_samples, num_inference_steps=50, guidance_scale=7.5).images
    all_images.extend(images)
    return all_images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="prompt")
            samples = gr.Slider(label="Samples",value=1)
            run = gr.Button(value="Run")
        with gr.Column():
            gallery = gr.Gallery(show_label=False)

    run.click(inference, inputs=[prompt,samples], outputs=gallery)
    gr.Examples([["a photo of sks toy riding a bicycle", 1,1]], [prompt,samples], gallery, inference, cache_examples=False)


## Unhash the line below to launch your demo in Gradio

# demo.launch()


In [ ]:
#@title Run the Stable Diffusion pipeline to generate quick samples in the Notebook

prompt = "a photo of a sks cat toy riding a bicycle" #@param {type:"string"}

num_samples = 3  #@param {type:"number"}
num_rows = 3 #@param {type:"number"}

all_images = [] 
for _ in range(num_rows):
    images = pipe([prompt] * num_samples, num_inference_steps=75, guidance_scale=7.5, seed = 'random').images
    all_images.extend(images)

grid = image_grid(all_images, num_samples, num_rows)
grid 

# Convert your concept to a Stable Diffusion Checkpoint
Use the cell below to convert your model to classic Stable Diffusion `.ckpt` format. This can then be used with any Stable Diffusion fork, such as the Web UI available to run from the Gradient Notebook Stable Diffusion Runtime tile. 

In [ ]:
# Params for `convert_diffusers_to_original_stable_diffusion.py`:
# --model_path = Path to the model directory (default "dreambooth-concept") to convert.
# --checkpoint_path = Path to the output model.
# --half = store_true, Save weights in half precision/

# Get script and convert model
%cd ~/../notebooks
!git clone https://github.com/huggingface/diffusers

# template: use this if you have a custom concept
# !python diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py --model_path <path to dreambooth concept> --checkpoint_path <name/path of your new model>


# Send our "dreambooth-concept" directory as .ckpt to the Web UI
!python diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py --model_path dreambooth-concept/ --checkpoint_path stable-diffusion-webui/models/Stable-diffusion/db-concept.ckpt

%cd ~/../notebooks


# Use the Stable Diffusion Web UI

Now that our new checkpoint file is in the right place, we can run the cell below to setup and launch the Automatic1111 Stable Diffusion Web UI from this Notebook. Click the Gradio link after the setup completes to access the Web UI in your local browser. 

> Note: Be sure to check out the Textual Inversion notebook before continuing!

# Loading in the Stable Diffusion v1-5 models for the Web UI 

To make accessing the Stable Diffusion models easy and not take up any storage, we have added the Stable Diffusion models v1-5 as mountable public datasets! 

First, navigate to the "Data Sources" tab using the navigator on the far left of the page.

Next, click "Public" to switch into the Gradient Public Datasets, and scroll down until you find "stable-diffusion-classic" near the bottom of the list. 

Finally, click "mount" to make them accessible from the "datasets" directory. This directory is in the root folder, so access it with the path `~/../datasets/stable-diffusion-classic/v1-5-pruned-emaonly.ckpt`

In [ ]:
%cd stable-diffusion-webui
# launch the webui
!python launch.py --share --ckpt ../../datasets/stable-diffusion-classic/v1-5-pruned-emaonly.ckpt


##  Upload model to DreamBooth Concepts Library

Also explore the [DreamBooth Concepts Library](https://huggingface.co/sd-dreambooth-library) 

In [ ]:
#@title Save your newly created concept? you may save it privately to your personal profile or collaborate to the [library of concepts](https://huggingface.co/sd-dreambooth-library)?
# #@markdown If you wish your model to be avaliable for everyone, add it to the public library. If you prefer to use your model privately, add your own profile.

# save_concept = True #@param {type:"boolean"}
# #@markdown Once you save it you can use your concept by loading the model on any `from_pretrained` function
# name_of_your_concept = "Cat toy" #@param {type:"string"}
# where_to_save_concept = "public_library" #@param ["public_library", "privately_to_my_profile"]

# #@markdown `hf_token_write`: leave blank if you logged in with a token with `write access` in the [Initial Setup](#scrollTo=KbzZ9xe6dWwf). If not, [go to your tokens settings and create a write access token](https://huggingface.co/settings/tokens)
# hf_token_write = "" #@param {type:"string"}
# if(save_concept):
#   from slugify import slugify
#   from huggingface_hub import HfApi, HfFolder, CommitOperationAdd
#   from huggingface_hub import create_repo
#   from IPython.display import display_markdown
#   api = HfApi()
#   your_username = api.whoami()["name"]
#   pipe = StableDiffusionPipeline.from_pretrained(
#     args.output_dir,
#     torch_dtype=torch.float16,
#   ).to("cuda")
#   os.makedirs("fp16_model",exist_ok=True)
#   pipe.save_pretrained("fp16_model")

#   if(where_to_save_concept == "public_library"):
#     repo_id = f"sd-dreambooth-library/{slugify(name_of_your_concept)}"
#     #Join the Concepts Library organization if you aren't part of it already
#     !curl -X POST -H 'Authorization: Bearer '$hf_token -H 'Content-Type: application/json' https://huggingface.co/organizations/sd-dreambooth-library/share/SSeOwppVCscfTEzFGQaqpfcjukVeNrKNHX
#   else:
#     repo_id = f"{your_username}/{slugify(name_of_your_concept)}"
#   output_dir = args.output_dir
#   if(not hf_token_write):
#     with open(HfFolder.path_token, 'r') as fin: hf_token = fin.read();
#   else:
#     hf_token = hf_token_write 
  
#   images_upload = os.listdir("my_concept")
#   image_string = ""
#   #repo_id = f"sd-dreambooth-library/{slugify(name_of_your_concept)}"
#   for i, image in enumerate(images_upload):
#       image_string = f'''{image_string}![image {i}](https://huggingface.co/{repo_id}/resolve/main/concept_images/{image})
# '''
#   readme_text = f'''---
# license: mit
# ---
# ### {name_of_your_concept} on Stable Diffusion via Dreambooth
# #### model by {api.whoami()["name"]}
# This your the Stable Diffusion model fine-tuned the {name_of_your_concept} concept taught to Stable Diffusion with Dreambooth.
# It can be used by modifying the `instance_prompt`: **{instance_prompt}**

# You can also train your own concepts and upload them to the library by using [this notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_dreambooth_training.ipynb).
# And you can run your new concept via `diffusers`: [Colab Notebook for Inference](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/sd_dreambooth_inference.ipynb), [Spaces with the Public Concepts loaded](https://huggingface.co/spaces/sd-dreambooth-library/stable-diffusion-dreambooth-concepts)

# Here are the images used for training this concept:
# {image_string}
# '''
#   #Save the readme to a file
#   readme_file = open("README.md", "w")
#   readme_file.write(readme_text)
#   readme_file.close()
#   #Save the token identifier to a file
#   text_file = open("token_identifier.txt", "w")
#   text_file.write(instance_prompt)
#   text_file.close()
#   operations = [
#     CommitOperationAdd(path_in_repo="token_identifier.txt", path_or_fileobj="token_identifier.txt"),
#     CommitOperationAdd(path_in_repo="README.md", path_or_fileobj="README.md"),
#   ]
#   create_repo(repo_id,private=True, token=hf_token)
  
#   api.create_commit(
#     repo_id=repo_id,
#     operations=operations,
#     commit_message=f"Upload the concept {name_of_your_concept} embeds and token",
#     token=hf_token
#   )
#   api.upload_folder(
#     folder_path="fp16_model",
#     path_in_repo="",
#     repo_id=repo_id,
#     token=hf_token
#   )
#   api.upload_folder(
#     folder_path=save_path,
#     path_in_repo="concept_images",
#     repo_id=repo_id,
#     token=hf_token
#   )
# display_markdown(f'''## Your concept was saved successfully. [Click here to access it](https://huggingface.co/{repo_id})
# ''', raw=True)